In [35]:
# Import dependencies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [36]:
# Import datasets
testDataFrame = pd.read_csv("./datasets/mnist/mnistTest.csv")
trainDataFrame = pd.read_csv("./datasets/mnist/mnistTrain.csv")

In [37]:
# Load data into numpy arrays
testData = np.array(testDataFrame)
trainData = np.array(trainDataFrame)

In [341]:
# Define our training variables

# Epsilon is our step size in gradient descent
epsilon = 0.001

# The labels for the data l
labelsRaw = trainData[:,0]
labels = [[1 if label == i else 0 for i in range(10)] for label in labelsRaw]

# The input vector
input = trainData[:,1:]

# Dimension of the input space (28 * 28 = 784 pixels)
n = 784

# Dimension of hidden layer ( 4 nodes )
m = 4

# Dimension of output ( 10 possible digits to classify )
l = 10

# Size of training data
k = 60000

### Initialize the weights $w^1_{ij}$ and $ w^2_{kl}$

- $w^0_{ij}$ will be a linear map $w^0: \mathbb{R}^n \to \mathbb{R}^m$

- $w^1_{kl}$ will be a linear map $w^1: \mathbb{R}^m \to \mathbb{R}^l$ 

In [342]:
# Initialize weights for the input and the hidden layer
w0 = np.random.random([n, m])
w1 = np.random.random([m, l])

# Initialize biases
b0 = np.random.random([m])
b1 = np.random.random([l])


### Forward Propogation
Begin with vector input $v \in \mathbb{R}^n$ where $n = 28 * 28 = 784$. The value at the hidden layer before regularization

$$
  \alpha^0_i =
      \sum_{j} w^0_{ji} v_j + b^0_i
$$

where $w_{ij}$ is the weight of neuron $i$ going into neuron $j$ and $b$ is the bias. Let
 $\sigma$ be  some non linear activation function

The value at the output layer
$$
  \alpha^1_i = \sum_j w^1_{ji} \sigma (\alpha^0_j) + b^1_i
$$

Then we apply a softmax function to get our prediction probabilities at the output layer
$$
  p_i = \text{softmax} (\alpha^1_i)
$$

In [343]:
# Define activation function and softmax 
def sigmoid(x):
  return 1 / ( 1 + np.exp(-x))

def sigmoidPrime(x):
  return np.exp(-x) / ( 1 + np.exp(-x)) ** 2

def softmax(x):
  return np.array(np.exp(x) / np.sum(np.exp(x)))

In [344]:
# Numpy implementation
def forwardProp(w0, w1, b0, b1, input, i):
  # Caluculate hidden layer values
  a0 = w0.T.dot(input[i]) + b0

  # Apply relu as activation function
  z = np.array([sigmoid(x) for x in a0])

  # Calculate output layer values
  z = w1.T.dot(z) + b1

  # apply softmax
  return softmax(z), a0


##  Back Propogation

Define the error function as the squared difference between prediction and labelled value. Let $l_i$ define the label for input $i$
$$
  E(w^0, w^1) = \frac{1}{2} \sum_i (l_i - p_i)^2
$$
We want to minimize $E(w^0, w^1)$. Handle the $w^0$ and $w^1$ cases separately.


In [345]:
def error(l, p):
  if (len(p) != len(l)):
    raise ValueError("Lengths must be the same")

  a = [0.5 * (p[i] - l[i])**2 for i in range(len(p))]
  return np.sum(a)


In [346]:
# Given error and last values 
def backProp(w0, w1, b0, b1, p, label, a0 ):

  def z(k):
    return (p[k] - label[k]) * p[k] * (1 + p[k])


  dw1 = np.array([[- epsilon 
          * sigmoidPrime(a0[i]) 
          * z(j) 
          * a0[i] 
            for i in range(m)] 
              for j in range(l)])

  dw2 = np.array([[- epsilon 
            * sigmoidPrime(a0[j]) 
            * input[i] 
            * np.sum([z(k) * w1[j][k] for k in range(l)]) 
                for i in range(n)] 
                  for j in range(m)])
  print(dw2)    
  w1 = w1 + dw1.T
  w0 = w0 + dw2.T

In [347]:
#
#  One iteration
i = 0
p, a0 = forwardProp(w0, w1, b0, b1, input, i)

print('error before: ', error(labels[i], p))

backProp(w0, w1, b0, b1, p, labels[i], a0)


p_new, a0_new = forwardProp(w0, w1, b0, b1, input, i)

print('error after: ', error(labels[i], p_new))


error before:  0.49172405467448543
[[[-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  ...
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]]

 [[-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  ...
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]]

 [[-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  ...
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]]

 [[-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  ...
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]
  [-0. -0. -0. ... -0. -0. -0.]]]
error after:  0.49172405467448543



### Hidden layer weights $w^1$
$$
  \frac{\partial E}{\partial w^1_{ij}} = \sum_k \frac{\partial E}{\partial p_k} \frac{\partial p_k}{\partial \alpha^1_k} \frac{\partial \alpha^1_k}{w^1_{ij}}
$$

Calculate each factor
$$
\begin{aligned}
  \frac{\partial E}{\partial p_k} &= \frac{1}{2} (l_k - p_k) * (-1) \\
  &= (p_k - l_k) \\
  \\
  \frac{\partial p_k}{\partial \alpha^1_k} 
    
    &= \frac{\partial}{\partial \alpha^1_k} 
        \frac{e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l}} 
  \\
  & = e^{\alpha^1_k} 
    \bigg ( 
      \frac{ e^{\alpha^1_k} }{ 
        \big( 
          \sum_l e^{ \alpha^1_k } 
        \big)^2} +
      \frac{1}{ \sum_l e^{ \alpha^1_l } } 
    \bigg ) 
  \\
  & = \frac{ e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l} } 
    \bigg ( 
      1 + \frac{e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l}} 
    \bigg )
  \\
  & = p_k ( 1 + p_k)
  
\end{aligned}
$$
Finally the derivative wrt the weight
$$
\begin{align}
  \frac{ \partial \alpha^1_k }{ \partial w^1_{ij} } 
    &= 
    \frac{\partial}{\partial w^1_{ij}} 
    \bigg( 
      \sum_l w^1_{lk} \sigma ( \alpha^0_l ) + b^1_k
    \bigg)
  \\
&= \sum_l \sigma ' ( \alpha^0_l ) \cdot \delta_{kj} \delta_{il} \alpha_l = \delta_{kj} \sigma ' ( \alpha^0_i ) \alpha_i
\end{align}
$$
Where $\delta$ denotes the Kroneckor delta. Increment the hidden layer output weights proportional to the derivative of the error where $\epsilon$ is some proportionality factor
$$
\begin{align}

  \Delta w^1_{ij} &= - \epsilon \sum_k p_k(p_k - l_k)(1 + p_k) \delta_{kj} \alpha^0_i \sigma ' ( \alpha^0_i )
  \\
  &= - \epsilon p_j(p_j - l_j)(1 + p_j) \alpha^0_i \sigma ' ( \alpha^0_i )
\end{align}
$$

### Input layer weights $w^0$

$$
  \frac{ \partial E }{ \partial w^0_{ij} } = 
  \sum_k
    \frac{ \partial E }{ \partial p_k } 
    \frac{ \partial p_k }{ \partial \alpha^1_k } 
    \frac{ \partial \alpha^1_k }{ w^0_{ij} }
$$

Only the last term is different. Keeping $\sigma$ generic:
$$
\begin{align}
  \frac{\partial \alpha^1_k}{\partial w^0_{ij}} 
  
    &= \frac{ \partial }{ \partial w^0_{ij} } 
      \bigg (
        \sum_l w^1_{lk} \sigma (\alpha^0_l) + b^1_k
      \bigg)
    \\

     &= \sum_l 
      w^1_{lk} 
      \cdot
      \frac{ \partial }{ \partial w^0_{ij} }
        \sigma 
          \big( 
            \alpha^0_l
          \big ) 
    \\

    &= \sum_l 
        w^1_{lk} 
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \frac{ \partial }{ \partial w^0_{ij} }
          \bigg ( 
            \sum_{m} w^0_{ml} v_m + b^0_l
          \bigg )
    \\
    &= \sum_l
        w^1_{lk}
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \sum_m
        \delta_{im} \delta_{jl} v_m 
    \\
    &= \sum_l
        w^1_{lk}
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \delta_{jl} v_i
    \\
    &= w^1_{jk}
      \cdot 
      \sigma'(\alpha^0_j)
      \cdot
      v_i
        
\end{align}
$$

Combining the previously calculated terms we get
$$
\begin{align}

  \Delta w^0_{ij}
  &= 
  - \epsilon \cdot \sum_k
    (p_k - l_k)
    p_k
    (1 + p_k) 
    \cdot 
    w^1_{jk}
      \cdot 
      \sigma'(\alpha^0_j)
      \cdot
      v_i
  \\
  &= - \epsilon 
    \cdot \sigma'(\alpha^0_j) 
    v_i 
    \sum_k
     (p_k - l_k)
      p_k
      (1 + p_k) 
      w^1_{jk}
\end{align}
$$